In [1]:
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
import pathlib
from google.cloud import storage
import tensorflow_hub as hub
from io import BytesIO
from tensorflow.python.lib.io import file_io
import seaborn as sn
from sklearn.metrics import confusion_matrix
import math

In [3]:
# data
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_grouped_classes.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_4classes.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_augment2x_siebren.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_augment_siebren.csv", header=None)
train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_grouped_classes_balanced.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_4classes_balanced.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_elephant.csv", header=None)

# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_series_250_shuffle.csv", header=None)
# train_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_series_200_shuffle_4classes.csv", header=None)

val_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren.csv", header=None)
test_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren.csv", header=None)
CLASS_NAMES = train_data[0].unique()

train_path = "gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren.csv"
eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren.csv"

In [4]:
len(test_data)

43612

In [5]:
train_data.groupby(0).nunique()

,1
0,
Bird,4742
Blank,23956
Buffalo_African,4826
Cat_Golden,276
Chevrotain_Water,3461
Chimpanzee,4133
Civet_African_Palm,716
Duiker_Blue,23997
Duiker_Red,23945


In [6]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
IMG_CHANNELS = 3

BATCH_SIZE = 128
# 10 is a magic number tuned for local training of this dataset.
SHUFFLE_BUFFER = 10 * BATCH_SIZE
AUTOTUNE = tf.data.experimental.AUTOTUNE

VALIDATION_IMAGES = 370
VALIDATION_STEPS = VALIDATION_IMAGES // BATCH_SIZE

def load_dataset(csv_of_filenames, batch_size, training=True):    
    dataset = tf.data.TextLineDataset(filenames=csv_of_filenames) \
        .map(decode_csv, num_parallel_calls=tf.data.AUTOTUNE)
    
    if training:
        dataset = dataset \
            .map(read_and_preprocess_with_augment, num_parallel_calls=tf.data.AUTOTUNE) \
            .repeat(count=1)
    else:
        dataset = dataset \
            .map(read_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
            .repeat(count=1)  # Each photo used once.
    
    return dataset.batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE)

def decode_csv(csv_row):
    record_defaults = ["img", "path"]
    label_string, filename = tf.io.decode_csv(csv_row, record_defaults)
    image_bytes = tf.io.read_file(filename=filename)
    label = tf.math.equal(CLASS_NAMES, label_string)
    return image_bytes, label

MAX_DELTA = 63.0 / 255.0  # Change brightness by at most 17.7%
CONTRAST_LOWER = 0.2
CONTRAST_UPPER = 1.8

def read_and_preprocess(image_bytes, label, random_augment=False):
    img = decode_img(image_bytes, [IMG_WIDTH, IMG_HEIGHT])
    return img, label


def read_and_preprocess_with_augment(image_bytes, label):
    return read_and_preprocess(image_bytes, label, random_augment=True)

def decode_img(img, reshape_dims):
    # Convert the compressed string to a 3D uint8 tensor.
    img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # Resize the image to the desired size.
    return tf.image.resize(img, reshape_dims)

In [7]:
model = keras.models.load_model("gs://dataset-gabon/train_val_test_csv_Siebren/CNN_models/Xception/model.H5")

In [8]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

train_ds = load_dataset(train_path, BATCH_SIZE)
eval_ds = load_dataset(eval_path, BATCH_SIZE, training=False)

# Test model

In [9]:
eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren.csv"
# eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren_grouped_classes.csv"
# eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren_4classes.csv"
# eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren_grouped_classes_balanced2.csv"
# eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_4classes_siebren.csv"
# eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren_series_200_shuffle_4classes.csv"
# eval_path = "gs://dataset-gabon/train_val_test_csv_Siebren/val_data_siebren_4classes_balanced.csv"


eval_ds = load_dataset(eval_path, BATCH_SIZE, training=False)
test_loss, test_acc = model.evaluate(eval_ds, verbose=1)
print(test_acc)

476/476 [==============================] - 12294s 26s/step - loss: 0.7292 - accuracy: 0.7636
0.7629993557929993


In [19]:
# test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/train_data_siebren_4classes.csv"
# test_ds = load_dataset(test_path, BATCH_SIZE, training=False)

In [30]:
test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren.csv"
# # test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren_grouped_classes.csv"
# test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren_4classes.csv"
# # test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren_grouped_classes_balanced2.csv"
# test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_4classes_siebren.csv"
# test_path = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren_4classes_balanced.csv"

test_ds = load_dataset(test_path, BATCH_SIZE, training=False)

In [34]:
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(test_acc)

KeyboardInterrupt: 

In [31]:
PREDICTIONS = len(test_data)//BATCH_SIZE
y_test = []
counter = 0
for i in tqdm(test_ds):
    if counter < PREDICTIONS:
        counter += 1
        for j in i[1]:
            y_test.append(np.where(j == True)[0][0])
    else:
        break
print(len(y_test))

340it [01:05,  5.17it/s]

43520


In [ ]:
y_pred = model.predict(test_ds, steps=PREDICTIONS)
predicted_categories = tf.argmax(y_pred, axis=1)
true_categories = y_test

In [37]:
new_labels = CLASS_NAMES.copy()
report = classification_report(true_categories, predicted_categories, output_dict=True)
df = pd.DataFrame(report).transpose()
if new_labels[len(new_labels) -1] != "-":
    new_labels = np.append(new_labels, ["-", "-", "-"])
df = df.round(decimals=2)
df['label'] = new_labels
df["precision"].iloc[len(new_labels) -3] = "-"
df["recall"].iloc[len(new_labels) -3] = "-"
df["support"].iloc[len(new_labels) -3] = df["support"].iloc[len(new_labels) -2]
df

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,precision,recall,f1-score,support,label
0,0.75,0.51,0.60,2868.0,Rat_Giant
1,0.15,0.18,0.16,764.0,Squirrel
2,0.6,0.79,0.68,4976.0,Duiker_Red
3,0.11,0.07,0.09,338.0,Rodent
4,0.8,0.93,0.86,4994.0,Human
5,0.48,0.77,0.59,310.0,Guineafowl_Black
6,0.18,0.19,0.18,1009.0,Monkey
7,0.55,0.53,0.54,4998.0,Duiker_Blue
8,0.6,0.48,0.53,1089.0,Duiker_Yellow_Backed
9,0.7,0.89,0.78,4978.0,Elephant_African


In [16]:
macro_f1_score = []
sum_list_recall = []
sum_list_precision = []
for idx, item in df.iterrows():
    if idx != "accuracy" and idx != "macro avg" and idx != "weighted avg":
        if item["label"] == "Human" or item["label"] == "Elephant_African" or item["label"] == "Blank":
            if item["precision"] != 0 and item["recall"] != 0:
                macro_f1_score.append(1.5 * (item["precision"] * item["recall"]) / ((0.5*item["precision"]) + item["recall"]))
            else:
                macro_f1_score.append(0)
        else:
            sum_list_recall.append(item["recall"])
            sum_list_precision.append(item["precision"])

recall = sum(sum_list_recall) / len(sum_list_recall)
precision = sum(sum_list_precision) / len(sum_list_precision)
            
macro_f1_score.append(1.5 * (precision * recall) / ((0.5*precision) + recall))

print("Macro f0.5 score: ", round(sum(macro_f1_score) / len(macro_f1_score), 2))
print("Human f0.5 score: ", round(1.5 * (df[df["label"] == "Human"]["recall"][0] * df[df["label"] == "Human"]["precision"][0]) / ((0.5*df[df["label"] == "Human"]["precision"][0]) + df[df["label"] == "Human"]["recall"][0]), 2))
print("Other f0.5 score: ", round(1.5 * (df[df["label"] == "Other_animal"]["recall"][0] * df[df["label"] == "Other_animal"]["precision"][0]) / ((0.5*df[df["label"] == "Other_animal"]["precision"][0]) + df[df["label"] == "Other_animal"]["recall"][0]), 2))
try:
    print("Elephant f0.5 score: ", round(1.5 * (df[df["label"] == "Elephant_African"]["recall"][0] * df[df["label"] == "Elephant_African"]["precision"][0]) / ((0.5*df[df["label"] == "Elephant_African"]["precision"][0]) + df[df["label"] == "Elephant_African"]["recall"][0]), 2))
except:
    print("Elephant f0.5 score: ", 0.00)

Macro f0.5 score:  0.83
Human f0.5 score:  0.85
Other f0.5 score:  0.93
Elephant f0.5 score:  0.74


In [17]:
mean_aggregated_cross_entropy_loss = {}

for idx, pred in enumerate(y_pred):
    loss = -math.log(pred[true_categories[idx]])
    if CLASS_NAMES[true_categories[idx]] in mean_aggregated_cross_entropy_loss:
        mean_aggregated_cross_entropy_loss[CLASS_NAMES[true_categories[idx]]].append(loss)
    else:
        mean_aggregated_cross_entropy_loss[CLASS_NAMES[true_categories[idx]]] = [loss]

for keys, values in mean_aggregated_cross_entropy_loss.items():
    mean = sum(values)/ len(values)
    mean_aggregated_cross_entropy_loss[keys] = round(mean, 2)
    
# mean_aggregated_cross_entropy_loss

In [21]:
mean_aggregated_cross_entropy_loss_final = []
sum_values = []
for key, value in mean_aggregated_cross_entropy_loss.items():
    if key == "Human" or key == "Elephant_African" or key == "Blank":
        mean_aggregated_cross_entropy_loss_final.append(value)
    else:
        sum_values.append(value)

mean_aggregated_cross_entropy_loss_final.append(sum(sum_values) / len(sum_values))

print("mean aggregated cross entropy loss: ", round(sum(mean_aggregated_cross_entropy_loss_final) / len(mean_aggregated_cross_entropy_loss_final), 2))
print("coss entropy loss Human: ", mean_aggregated_cross_entropy_loss["Human"])
print("coss entropy loss Elephant: ", mean_aggregated_cross_entropy_loss["Elephant_African"])
print("coss entropy loss Other: ", mean_aggregated_cross_entropy_loss["Other_animal"])
print("coss entropy loss blank: ", mean_aggregated_cross_entropy_loss["Blank"])

mean aggregated cross entropy loss:  0.64
coss entropy loss Human:  0.48
coss entropy loss Elephant:  0.75
coss entropy loss Other:  0.21
coss entropy loss blank:  1.1


# MYANMAR

In [10]:
test_data_myan = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/test_data_myanmar_siebren.csv", header=None)
test_path_myan = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_myanmar_siebren.csv"
test_ds_myan = load_dataset(test_path_myan, BATCH_SIZE, training=False)

In [11]:
test_loss, test_acc = model.evaluate(test_ds_myan, verbose=1)
print(test_acc)

     17/Unknown - 515s 30s/step - loss: 0.6855 - accuracy: 0.7693

KeyboardInterrupt: 

In [19]:
PREDICTIONS = len(test_data_myan)//BATCH_SIZE
y_test = []
counter = 0
for i in tqdm(test_ds_myan):
    if counter < PREDICTIONS:
        counter += 1
        for j in i[1]:
            y_test.append(np.where(j == True)[0][0])
    else:
        break
print(len(y_test))

36it [05:48,  9.69s/it]

4608


In [20]:
y_pred = model.predict(test_ds_myan, steps=PREDICTIONS)
predicted_categories = tf.argmax(y_pred, axis=1)
true_categories = y_test

In [21]:
new_labels = CLASS_NAMES.copy()
report = classification_report(true_categories, predicted_categories, output_dict=True)
df = pd.DataFrame(report).transpose()
a = pd.Series(list(new_labels))
b = list(df.index[:-3])
b = list(map(int, b))
new_labels = list(a[b])
if new_labels[len(new_labels) -1] != "-":
    new_labels = np.append(new_labels, ["-", "-", "-"])
df = df.round(decimals=2)
df['label'] = new_labels
df["precision"].iloc[len(new_labels) -3] = "-"
df["recall"].iloc[len(new_labels) -3] = "-"
df["support"].iloc[len(new_labels) -3] = df["support"].iloc[len(new_labels) -2]
df

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning

,precision,recall,f1-score,support,label
0,0.96,0.74,0.84,2047.0,Human
1,0.0,0.0,0.00,0.0,Hog_Red_River
2,0.87,0.87,0.87,2495.0,Blank
3,0.0,0.0,0.00,0.0,Buffalo_African
4,0.0,0.0,0.00,0.0,Duiker_Blue
5,0.12,0.71,0.20,66.0,Elephant_African
9,0.0,0.0,0.00,0.0,Chimpanzee
13,0.0,0.0,0.00,0.0,Duiker_Red
14,0.0,0.0,0.00,0.0,Bird
16,0.0,0.0,0.00,0.0,Leopard_African


In [22]:
macro_f1_score = []
sum_list_recall = []
sum_list_precision = []
for idx, item in df.iterrows():
    if idx != "accuracy" and idx != "macro avg" and idx != "weighted avg":
        if item["label"] == "Human" or item["label"] == "Elephant_African" or item["label"] == "Blank":
            if item["precision"] != 0 and item["recall"] != 0:
                macro_f1_score.append(1.5 * (item["precision"] * item["recall"]) / ((0.5*item["precision"]) + item["recall"]))
            else:
                macro_f1_score.append(0)

print("Macro f0.5 score: ", round(sum(macro_f1_score) / len(macro_f1_score), 2))
print("Human f0.5 score: ", round(1.5 * (df[df["label"] == "Human"]["recall"][0] * df[df["label"] == "Human"]["precision"][0]) / ((0.5*df[df["label"] == "Human"]["precision"][0]) + df[df["label"] == "Human"]["recall"][0]), 2))
try:
    print("Elephant f0.5 score: ", round(1.5 * (df[df["label"] == "Elephant_African"]["recall"][0] * df[df["label"] == "Elephant_African"]["precision"][0]) / ((0.5*df[df["label"] == "Elephant_African"]["precision"][0]) + df[df["label"] == "Elephant_African"]["recall"][0]), 2))
except:
    print("Elephant f0.5 score: ", 0.00)

Macro f0.5 score:  0.64
Human f0.5 score:  0.87
Elephant f0.5 score:  0.17


In [23]:
mean_aggregated_cross_entropy_loss = {}

for idx, pred in enumerate(y_pred):
    loss = -math.log(pred[true_categories[idx]])
    if CLASS_NAMES[true_categories[idx]] in mean_aggregated_cross_entropy_loss:
        mean_aggregated_cross_entropy_loss[CLASS_NAMES[true_categories[idx]]].append(loss)
    else:
        mean_aggregated_cross_entropy_loss[CLASS_NAMES[true_categories[idx]]] = [loss]

for keys, values in mean_aggregated_cross_entropy_loss.items():
    mean = sum(values)/ len(values)
    mean_aggregated_cross_entropy_loss[keys] = round(mean, 2)
    
# mean_aggregated_cross_entropy_loss

In [24]:
mean_aggregated_cross_entropy_loss_final = []
for key, value in mean_aggregated_cross_entropy_loss.items():
    mean_aggregated_cross_entropy_loss_final.append(value)

print("mean aggregated cross entropy loss: ", round(sum(mean_aggregated_cross_entropy_loss_final)/ len(mean_aggregated_cross_entropy_loss_final), 2))
print("coss entropy loss Human: ", mean_aggregated_cross_entropy_loss["Human"])
print("coss entropy loss Elephant: ", mean_aggregated_cross_entropy_loss["Elephant_African"])

mean aggregated cross entropy loss:  0.65
coss entropy loss Human:  0.86
coss entropy loss Elephant:  0.67


# SERENGETI

In [44]:
human = 4
elephant = 9
blank = 12
grouped = 28

In [21]:
test_data_serengeti = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/serengeti_siebren.csv", header=None)
test_path_serengeti = "gs://dataset-gabon/train_val_test_csv_Siebren/serengeti_siebren.csv"
test_ds_serengeti = load_dataset(test_path_serengeti, BATCH_SIZE, training=False)

In [22]:
# test_loss, test_acc = model.evaluate(test_ds_serengeti, verbose=1)
# print(test_acc)

In [42]:
PREDICTIONS2 = len(test_data_serengeti)//BATCH_SIZE
y_test2 = []
counter2 = 0

for i in tqdm(test_ds_serengeti):
    if counter2 < PREDICTIONS2:
        counter2 += 1
        for j in i[1]:
            if True in j:
                y_test2.append(np.where(j == True)[0][0])
            else:
                y_test2.append(grouped)
#                 y_test.append(10) # <- grouped
    else:
        break
print(len(y_test2))

82it [06:53,  5.04s/it]

10496


In [59]:
y_pred = model.predict(test_ds_serengeti, steps=PREDICTIONS2)
predicted_categories2 = tf.argmax(y_pred, axis=1)
true_categories = y_test2

In [40]:
predicted_categories = []
for i in predicted_categories2:
#     if i.numpy() != 4 and i.numpy() != 9 and i.numpy() != 12:
#         predicted_categories.append(28)
#     if i.numpy() != 3 and i.numpy() != 4 and i.numpy() != 9: # <-- augmented
#         predicted_categories.append(28)
#     if i.numpy() != 2 and i.numpy() != 5 and i.numpy() != 6: # <-- grouped  
#         predicted_categories.append(10)
#     if i.numpy() != 13 and i.numpy() != 24 and i.numpy() != 25: # <-- augmented - balanced 
#         predicted_categories.append(28)
    if i.numpy() != human and i.numpy() != elephant and i.numpy() != blank: # <-- no series
        predicted_categories.append(grouped)
    else:
        predicted_categories.append(i.numpy())
# predicted_categories = predicted_categories2 # <- 4classes

In [61]:
new_labels = CLASS_NAMES.copy()
new_labels = np.append(new_labels, "Other_animal")
report = classification_report(true_categories, predicted_categories, output_dict=True)
df = pd.DataFrame(report).transpose()
print(df["precision"]["accuracy"])
a = pd.Series(list(new_labels))
b = list(df.index[:-3])
b = list(map(int, b))
new_labels = list(a[b])
if new_labels[len(new_labels) -1] != "-":
    new_labels = np.append(new_labels, ["-", "-", "-"])
df = df.round(decimals=2)
df['label'] = new_labels
df["precision"].iloc[len(new_labels) -3] = "-"
df["recall"].iloc[len(new_labels) -3] = "-"
df["support"].iloc[len(new_labels) -3] = df["support"].iloc[len(new_labels) -2]
df

0.7235137195121951


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,precision,recall,f1-score,support,label
4,0.76,0.82,0.79,2680.0,Human
9,0.74,0.83,0.78,2701.0,Elephant_African
12,0.67,0.71,0.69,2343.0,Blank
28,0.7,0.54,0.61,2772.0,Other_animal
accuracy,-,-,0.72,10496.0,-
macro avg,0.72,0.72,0.72,10496.0,-
weighted avg,0.72,0.72,0.72,10496.0,-


In [32]:
macro_f1_score = []
sum_list_recall = []
sum_list_precision = []
for idx, item in df.iterrows():
    if idx != "accuracy" and idx != "macro avg" and idx != "weighted avg":
        if item["label"] == "Human" or item["label"] == "Elephant_African" or item["label"] == "Blank":
            if item["precision"] != 0 and item["recall"] != 0:
                macro_f1_score.append(1.5 * (item["precision"] * item["recall"]) / ((0.5*item["precision"]) + item["recall"]))
            else:
                macro_f1_score.append(0)
        else:
            sum_list_recall.append(item["recall"])
            sum_list_precision.append(item["precision"])

recall = sum(sum_list_recall) / len(sum_list_recall)
precision = sum(sum_list_precision) / len(sum_list_precision)
            
macro_f1_score.append(1.5 * (precision * recall) / ((0.5*precision) + recall))

print("Macro f0.5 score: ", round(sum(macro_f1_score) / len(macro_f1_score), 2))
print("Human f0.5 score: ", round(1.5 * (df[df["label"] == "Human"]["recall"][0] * df[df["label"] == "Human"]["precision"][0]) / ((0.5*df[df["label"] == "Human"]["precision"][0]) + df[df["label"] == "Human"]["recall"][0]), 2))
try:
    print("Elephant f0.5 score: ", round(1.5 * (df[df["label"] == "Elephant_African"]["recall"][0] * df[df["label"] == "Elephant_African"]["precision"][0]) / ((0.5*df[df["label"] == "Elephant_African"]["precision"][0]) + df[df["label"] == "Elephant_African"]["recall"][0]), 2))
except:
    print("Elephant f0.5 score: ", 0.00)

Macro f0.5 score:  0.35
Human f0.5 score:  0.62
Elephant f0.5 score:  0.0


In [33]:
NEW_CLASS_NAMES = np.append(CLASS_NAMES, "Other_animal")

In [34]:
mean_aggregated_cross_entropy_loss = {}

for idx, pred in enumerate(y_pred):
#     if true_categories[idx] == 28:
#         arr = np.delete(pred, [4,9,12])
#         loss = -math.log(arr[tf.argmax(arr).numpy()])
#     if true_categories[idx] == 10: # <-- grouped
#         arr = np.delete(pred, [2,5,6])
#         loss = -math.log(arr[tf.argmax(arr).numpy()])
    if true_categories[idx] == grouped: # <-- no series
        arr = np.delete(pred, [human,elephant,blank])
        loss = -math.log(arr[tf.argmax(arr).numpy()])
    else:    
        loss = -math.log(pred[true_categories[idx]])
    if NEW_CLASS_NAMES[true_categories[idx]] in mean_aggregated_cross_entropy_loss:
        mean_aggregated_cross_entropy_loss[NEW_CLASS_NAMES[true_categories[idx]]].append(loss)
    else:
        mean_aggregated_cross_entropy_loss[NEW_CLASS_NAMES[true_categories[idx]]] = [loss]

for keys, values in mean_aggregated_cross_entropy_loss.items():
    mean = sum(values)/ len(values)
    mean_aggregated_cross_entropy_loss[keys] = round(mean, 2)
    
# mean_aggregated_cross_entropy_loss

In [35]:
mean_aggregated_cross_entropy_loss_final = []
for key, value in mean_aggregated_cross_entropy_loss.items():
    mean_aggregated_cross_entropy_loss_final.append(value)

print("mean aggregated cross entropy loss: ", round(sum(mean_aggregated_cross_entropy_loss_final)/ len(mean_aggregated_cross_entropy_loss_final), 2))
print("coss entropy loss Human: ", mean_aggregated_cross_entropy_loss["Human"])
print("coss entropy loss Elephant: ", mean_aggregated_cross_entropy_loss["Elephant_African"])

mean aggregated cross entropy loss:  1.03
coss entropy loss Human:  0.62
coss entropy loss Elephant:  0.87


In [17]:
def show_img(filename):
    img = tf.io.read_file(filename=filename)
    # Convert the compressed string to a 3D uint8 tensor.
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    plt.figure(figsize = (16,16))
    plt.imshow(img)
    return img

In [18]:
# import warnings

In [19]:
# plt.ion()
# for idx, i in enumerate(true_categories):
#     if i == 0:
#         if i == predicted_categories[idx]:
#             with warnings.catch_warnings():
#                 # this will suppress all warnings in this block
#                 warnings.simplefilter("ignore")
#                 show_img(test_data_myan[1][idx])
#     #         print("True ", CLASS_NAMES[i])
# #             print("Pred ", CLASS_NAMES[predicted_categories[idx]])

In [20]:
print(y_pred)

NameError: name 'y_pred' is not defined

In [ ]:
predict_list= []
for idx, i in enumerate(predicted_categories):
    if i == 3:
        if i == true_categories[idx]:
            predict_list.append(y_pred[idx])

elephant_c = []
other_c = []
blank_c= []
human_c = []
for i in predict_list:
    elephant_c.append(i[0])
    other_c.append(i[1])
    blank_c.append(i[2])
    human_c.append(i[3])
    
predict_list= []
for idx, i in enumerate(predicted_categories):
    if i == 3:
        if i != true_categories[idx]:
            predict_list.append(y_pred[idx])

elephant_i = []
other_i = []
blank_i= []
human_i = []
for i in predict_list:
    elephant_i.append(i[0])
    other_i.append(i[1])
    blank_i.append(i[2])
    human_i.append(i[3])

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt

objects = ('Elephant_African', 'Other_animal', 'Blank', 'Human')
y_pos = np.arange(len(objects))
performance_c = [sum(elephant_c) / len(elephant_c),sum(other_c) / len(other_c),sum(blank_c) / len(blank_c),sum(human_c) / len(human_c)]
performance_i = [sum(elephant_i) / len(elephant_i),sum(other_i) / len(other_i),sum(blank_i) / len(blank_i),sum(human_i) / len(human_i)]

# # plt.bar(y_pos, performance, align='center', alpha=0.5)
# plt.xticks(y_pos, objects)
# plt.ylabel('Probability')
# # plt.title('Incorrect predictions elephant')
# plt.ylim([0, 1])
# plt.show()


fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax.set_title('Full view')
ax.bar(y_pos, performance_c, align='center', alpha=0.5)

ax.set_ylim([0, 1])

ax2.set_title('Truncated view')
ax2.bar(y_pos, performance_i, align='center', alpha=0.5)

ax2.set_ylim([0, 1])

plt.show()

# todo CHECK WITH LESS HUMANS

In [59]:
test_data = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/test_data_siebren_4classes.csv", header=None)

In [60]:
human = test_data[test_data[0] == "Other_animal"].sample(5000)

In [61]:
rest = test_data[test_data[0] != "Other_animal"]

In [62]:
new_test = rest.append(human)

In [63]:
new_test = new_test.sample(frac=1)

In [64]:
new_test.groupby(0).nunique()

,1
0,
Blank,5000
Elephant_African,5000
Human,5000
Other_animal,5000


In [65]:
new_test.to_csv("gs://dataset-gabon/train_val_test_csv_Siebren/test_data_human.csv", header=None, index=None)

In [66]:
test_path_myan = "gs://dataset-gabon/train_val_test_csv_Siebren/test_data_human.csv"
test_ds_myan = load_dataset(test_path_myan, BATCH_SIZE, training=False)
# test_loss, test_acc = model.evaluate(test_ds_myan, verbose=1)
# print(test_acc)

In [67]:
test_data_myan = pd.read_csv("gs://dataset-gabon/train_val_test_csv_Siebren/test_data_human.csv", header=None)

In [68]:
PREDICTIONS = len(test_data_myan)//BATCH_SIZE
y_test = []
counter = 0
for i in tqdm(test_ds_myan):
    counter += 1
    for j in i[1]:
        y_test.append(np.where(j == True)[0][0])
print(len(y_test))

157it [02:35,  1.01it/s]

20000


In [69]:
# y_pred = model.predict(test_ds_myan)
# predicted_categories = []
# true_categories = []
# for idx, i in enumerate(y_pred):
#     if np.amax(i) > 0.9:
#         predicted_categories.append(np.argmax(i))
#         true_categories.append(y_test[idx])

In [70]:
# new_labels = CLASS_NAMES.copy()
# report = classification_report(true_categories, predicted_categories, output_dict=True)
# df = pd.DataFrame(report).transpose()
# a = pd.Series(list(new_labels))
# b = list(df.index[:-3])
# b = list(map(int, b))
# new_labels = list(a[b])
# if new_labels[len(new_labels) -1] != "-":
#     new_labels = np.append(new_labels, ["-", "-", "-"])
# df = df.round(decimals=2)
# df['label'] = new_labels
# df["precision"].iloc[len(new_labels) -3] = "-"
# df["recall"].iloc[len(new_labels) -3] = "-"
# df["support"].iloc[len(new_labels) -3] = df["support"].iloc[len(new_labels) -2]
# df

In [71]:
y_pred = model.predict(test_ds_myan)
predicted_categories = tf.argmax(y_pred, axis=1)
true_categories = y_test

In [72]:
new_labels = CLASS_NAMES.copy()
report = classification_report(true_categories, predicted_categories, output_dict=True)
df = pd.DataFrame(report).transpose()
a = pd.Series(list(new_labels))
b = list(df.index[:-3])
b = list(map(int, b))
new_labels = list(a[b])
if new_labels[len(new_labels) -1] != "-":
    new_labels = np.append(new_labels, ["-", "-", "-"])
df = df.round(decimals=2)
df['label'] = new_labels
df["precision"].iloc[len(new_labels) -3] = "-"
df["recall"].iloc[len(new_labels) -3] = "-"
df["support"].iloc[len(new_labels) -3] = df["support"].iloc[len(new_labels) -2]
df

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,precision,recall,f1-score,support,label
0,0.66,0.94,0.77,5000.0,Other_animal
1,0.93,0.84,0.88,5000.0,Human
2,0.88,0.76,0.81,5000.0,Elephant_African
3,0.91,0.73,0.81,5000.0,Blank
accuracy,-,-,0.82,20000.0,-
macro avg,0.84,0.82,0.82,20000.0,-
weighted avg,0.84,0.82,0.82,20000.0,-


In [73]:
# predict_list= []
# for idx, i in enumerate(predicted_categories):
#     if i == 1:
#         if i == true_categories[idx]:
#             predict_list.append(y_pred[idx])

# elephant_c = []
# other_c = []
# blank_c= []
# human_c = []
# for i in predict_list:
#     elephant_c.append(i[0])
#     other_c.append(i[1])
#     blank_c.append(i[2])
#     human_c.append(i[3])
    
# predict_list= []
# for idx, i in enumerate(predicted_categories):
#     if i == 1:
#         if i != true_categories[idx]:
#             predict_list.append(y_pred[idx])

# elephant_i = []
# other_i = []
# blank_i= []
# human_i = []
# for i in predict_list:
#     elephant_i.append(i[0])
#     other_i.append(i[1])
#     blank_i.append(i[2])
#     human_i.append(i[3])

In [74]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
# import numpy as np
# import matplotlib.pyplot as plt

# objects = ('Elephant_African', 'Other_animal', 'Blank', 'Human')
# y_pos = np.arange(len(objects))
# performance_c = [sum(elephant_c) / len(elephant_c),sum(other_c) / len(other_c),sum(blank_c) / len(blank_c),sum(human_c) / len(human_c)]
# performance_i = [sum(elephant_i) / len(elephant_i),sum(other_i) / len(other_i),sum(blank_i) / len(blank_i),sum(human_i) / len(human_i)]

# # # plt.bar(y_pos, performance, align='center', alpha=0.5)
# # plt.xticks(y_pos, objects)
# # plt.ylabel('Probability')
# # # plt.title('Incorrect predictions elephant')
# # plt.ylim([0, 1])
# # plt.show()


# fig = plt.figure(figsize=(12, 6))

# ax = fig.add_subplot(121)
# ax2 = fig.add_subplot(122)

# ax.set_title('Correct predictions human')
# ax.bar(y_pos, performance_c, align='center', alpha=0.5)
# ax.set_xticks([0,1,2,3]) 
# ax.set_xticklabels(['Other_animal','Human','Elephant_African','Blank'])

# ax.set_ylim([0, 1])

# ax2.set_title('Incorrect predictions human')
# ax2.bar(y_pos, performance_i, align='center', alpha=0.5)
# ax2.set_xticks([0,1,2,3]) 
# ax2.set_xticklabels(['Other_animal','Human','Elephant_African','Blank'])

# ax2.set_ylim([0, 1])

# plt.show()

In [75]:
# elephant = []
# other = []
# blank= []
# human = []
# for i in predicted_categories:
#     if i == 0:
#         other.append(i)
#     if i == 1:
#         human.append(i)
#     if i == 2:
#         elephant.append(i)
#     if i == 3:
#         blank.append(i)

In [77]:
other = 0
human = 0
elephant = 0 
blank = 0
for idx, i in enumerate(predicted_categories):
    if i == 0:
        other += 1
    if i == 1:
        human += 1
    if i == 2:
        elephant += 1
    if i == 3:
        blank += 1

In [78]:
print(f"other: {other}, human: {human}, elephant: {elephant}, blank: {blank}")

other: 7140, human: 4507, elephant: 4332, blank: 4021


true_human: 100, pred_human: 1589, tp_human: 84, fp_human: 1505, fn_human: 16

true_human: 500, pred_human: 1929, tp_human: 424, fp_human: 1505, fn_human: 76

true_human: 1000, pred_human: 2362, tp_human: 857, fp_human: 1505, fn_human: 143

true_human: 2500, pred_human: 3625, tp_human: 2120, fp_human: 1505, fn_human: 380

true_human: 5000, pred_human: 5761, tp_human: 4256, fp_human: 1505, fn_human: 744

In [40]:
fp_list = {0:0, 1:0, 3:0}
for idx, i in enumerate(predicted_categories):
    if i == 2 and true_categories[idx] != 2:
        fp_list[true_categories[idx]] += 1

In [41]:
fp_list

{0: 1197, 1: 191, 3: 117}

In [80]:
Balanced
Human &	24104 & 5000 & 4835 \\
Blank &	23956 & 5000 & 5000 \\
Elephant African & 24054 & 5000 & 5376 \\
Other animal & 171514 & 5000 & 4835 \\

In [ ]:
Human &	24104 & 5000 & 4507 \\
Blank &	23956 & 5000 & 4021 \\
Elephant African & 24054 & 5000 & 4332 \\
Other animal & 171514 & 5000 & 7140 \\